# 행정안전부 도로명주소 API
### 건물명으로부터 도로명주소 구하기

In [4]:
import requests
from urllib.parse import urlparse, quote
import pandas as pd

In [2]:
key_fd = open('roadapikey.txt', mode='r')
roadapi_key = key_fd.read(100)
key_fd.close()

In [3]:
road_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
url = f'{road_url}?confmKey={roadapi_key}&currentPage=1&countPerPage=10&keyword=강서구청&resultType=json'
urlparse(url)

ParseResult(scheme='https', netloc='www.juso.go.kr', path='/addrlink/addrLinkApi.do', params='', query='confmKey=U01TX0FVVEgyMDIwMTExOTEwMjMzNTExMDQzNzg=&currentPage=1&countPerPage=10&keyword=강서구청&resultType=json', fragment='')

In [5]:
quote('강서구청')

'%EA%B0%95%EC%84%9C%EA%B5%AC%EC%B2%AD'

In [7]:
url = f'{road_url}?confmKey={roadapi_key}&currentPage=1&countPerPage=10&keyword={quote("강서구청")}&resultType=json'
url

'https://www.juso.go.kr/addrlink/addrLinkApi.do?confmKey=U01TX0FVVEgyMDIwMTExOTEwMjMzNTExMDQzNzg=&currentPage=1&countPerPage=10&keyword=%EA%B0%95%EC%84%9C%EA%B5%AC%EC%B2%AD&resultType=json'

In [8]:
result = requests.get(url).json()
result

{'results': {'common': {'errorMessage': '정상',
   'countPerPage': '10',
   'totalCount': '6',
   'errorCode': '0',
   'currentPage': '1'},
  'juso': [{'detBdNmList': '',
    'engAddr': '302, Hwagok-ro, Gangseo-gu, Seoul',
    'rn': '화곡로',
    'emdNm': '화곡동',
    'zipNo': '07658',
    'roadAddrPart2': ' (화곡동)',
    'emdNo': '02',
    'sggNm': '강서구',
    'jibunAddr': '서울특별시 강서구 화곡동 980-16 강서구청',
    'siNm': '서울특별시',
    'roadAddrPart1': '서울특별시 강서구 화곡로 302',
    'bdNm': '강서구청',
    'admCd': '1150010300',
    'udrtYn': '0',
    'lnbrMnnm': '980',
    'roadAddr': '서울특별시 강서구 화곡로 302 (화곡동)',
    'lnbrSlno': '16',
    'buldMnnm': '302',
    'bdKdcd': '0',
    'liNm': '',
    'rnMgtSn': '115003005069',
    'mtYn': '0',
    'bdMgtSn': '1150010300109800016015086',
    'buldSlno': '0'},
   {'detBdNmList': '어린이집',
    'engAddr': '72, Hwagok-ro 44na-gil, Gangseo-gu, Seoul',
    'rn': '화곡로44나길',
    'emdNm': '화곡동',
    'zipNo': '07663',
    'roadAddrPart2': ' (화곡동)',
    'emdNo': '01',
    'sggNm': '강

In [9]:
addr_list = result['results']['juso']
len(addr_list)

6

In [10]:
for addr in addr_list:
    print(addr['bdNm'], addr['roadAddr'])

강서구청 서울특별시 강서구 화곡로 302 (화곡동)
강서구청 서울특별시 강서구 화곡로44나길 72 (화곡동)
강서구청 부산광역시 강서구 낙동북로 477 (대저1동)
강서구청청소차고 서울특별시 강서구 남부순환로 208 (외발산동)
강서구청가양동별관 서울특별시 강서구 양천로59길 38 (가양동)
강서구청역 부산광역시 강서구 낙동북로 485 (대저1동)


### 강서구청, 양천구청, 부천시청, 영등포구청, 김포시청

In [11]:
bldg_list = ['강서구청', '양천구청', '부천시청', '영등포구청', '김포시청']
road_addr_list = []
for bldg in bldg_list:
    url = f'{road_url}?confmKey={roadapi_key}&currentPage=1&countPerPage=10&keyword={quote(bldg)}&resultType=json'
    results = requests.get(url).json()
    addr_list = results['results']['juso']
    road_addr = addr_list[0]['roadAddr']
    road_addr_list.append(road_addr)
    print(road_addr)

서울특별시 강서구 화곡로 302 (화곡동)
서울특별시 양천구 목동동로 105 (신정동)
경기도 부천시 길주로 210-2 (중동)
서울특별시 영등포구 당산로 123 (당산동3가)
경기도 김포시 사우중로 1 (사우동)


In [12]:
df = pd.DataFrame({
                'bldg': bldg_list,
                'road_addr': road_addr_list
})
df

,bldg,road_addr
0,강서구청,서울특별시 강서구 화곡로 302 (화곡동)
1,양천구청,서울특별시 양천구 목동동로 105 (신정동)
2,부천시청,경기도 부천시 길주로 210-2 (중동)
3,영등포구청,서울특별시 영등포구 당산로 123 (당산동3가)
4,김포시청,경기도 김포시 사우중로 1 (사우동)


In [13]:
df.to_csv('address.csv', sep=',', encoding='utf8', index=False)